<a href="https://colab.research.google.com/github/PQlavka/YaMarketParser/blob/main/YaM_Counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Виктор Остапюк


In [ ]:
import requests
from bs4 import BeautifulSoup
from itertools import groupby

def load_user_data(page, session):
  # тут можно поменять фильтры
  url = '%s&onstock=0&offer-shipping=pickup&delivery-interval=5&page=%d&local-offers-first=1' % (main_link, page)
  request = session.get(url)
  #print(url)
  return request.text

def contain_needed_obj(text):
  soup = BeautifulSoup(text)
  obj_list = soup.find('article', {'data-autotest-id': 'product-snippet'})
  return obj_list is not None

def not_end_of_list(text):
  soup = BeautifulSoup(text)
  endol = soup.find('a', {'aria-label': 'Следующая страница'})
  return endol is not None

def other_regions_span(text):
  soup = BeautifulSoup(text)
  span = soup.find('span', {'class': '_3J6m98nXNU'})
  return span is not None

def load_objects_names(dict, data):
  for obj in data:
    dict.append(obj.find('span', {'data-tid' : 'ce80a508'}).contents[0])


## **Парсер для Я.Маркет**
* `unique` - `list[string]` - список уникальных элементов
* `doubled` - `list[int]` - список индексов повторяющихся значений
* `obj_dict` - `list[string]` - список всех элементов
* `total` - `int` - общее кол-во элементов 

### Как использовать
значение `main_link` заменить на свое (ссылка на каталог Я.Маркета должна оканчиваться атрибутом lr (lr=50))

Cкопировать из браузера ("Сеть" в "Инструментах разработчика", самый продолжительный запрос -> заголовки запроса) значения User-Agent и Cookie в `s.headers.update(...)`

Запустить всё по порядку (далее запускать только обновление `main_link` и применение парсера)

Для выгрузки в excel использовать последний блок

**Внимание!**

Запросы рекомендую выполнять несколько раз (от 3). Парсер может дать осечку и выдать меньший результат, чем есть на деле (например: 120 вместо 670). Стоит работать с наибольшим результатом из 3-ех попыток. Причина в капче :(


In [ ]:
main_link = 'https://market.yandex.ru/catalog--mobilnye-telefony-v-permi/54726/list?hid=91491&lr=50'

In [ ]:
s = requests.Session() 

In [ ]:
# "Сеть" в "Инструментах разработчика", самый продолжительный запрос -> заголовки запроса
s.headers.update({
        'User-Agent': '',
        'Cookie': ''
    })

In [ ]:
# применение парсера
obj_dict = [] # конечный результат в виде массива
page = 1
total = 0
while True:
  data = load_user_data(page, s)
  if (contain_needed_obj(data) and not_end_of_list(data)):
    soup = BeautifulSoup(data)
    if other_regions_span(data):
      span = soup.find('span', {'class': '_3J6m98nXNU'})
      obj_list = span.find_all_previous('article', {'data-autotest-id': 'product-snippet'})
      load_objects_names(obj_dict, obj_list)
      total+=len(obj_list) 
      break
  else:
    obj_list = soup.find_all('article', {'data-autotest-id': 'product-snippet'})
    load_objects_names(obj_dict, obj_list)
    page+=1 
    total+=len(obj_list) 
  else:
    break

doubled = [i for i, x in enumerate(obj_dict) if obj_dict.count(x) > 1] # повторяющиеся элементы
unique = list(dict.fromkeys(obj_dict)) # уникальные элементы

print("total: ", total)
print("doubled: ", len(doubled))
print("unique: ", len(unique))



total:  475
doubled:  112
unique:  411


## **Выгрузка списка уникальных элементов в excel**

In [ ]:

unique_sorted = sorted(unique)

import pandas as pd

df = pd.DataFrame() 
df['Name - YaM'] = unique_sorted[0::1] 
  
# Converting to excel 
df.to_excel('result.xlsx', index = False) 